In [1]:
import pandas as pd

In [2]:
prices = pd.DataFrame(data=[
    {'ticker': 'AAPL', 'date': '2022-01-01', 'price': 100},
    {'ticker': 'PLTR', 'date': '2022-01-02', 'price': 101},
    {'ticker': 'MSFT', 'date': '2022-01-03', 'price': 102},
])
prices

,ticker,date,price
0,AAPL,2022-01-01,100
1,PLTR,2022-01-02,101
2,MSFT,2022-01-03,102


In [23]:
prices.set_index('ticker')['price'].get('A')

In [25]:
sorted(prices.set_index('ticker')['price'].items(), key=lambda x: x[1], reverse=True)

[('MSFT', 102), ('PLTR', 101), ('AAPL', 100)]

## FinViz stats

In [4]:
import aiohttp
import pandas as pd
from io import StringIO

from config import settings


async def get_ticker_info(tickers: list[str]):
    async with aiohttp.ClientSession() as session:
        data = await session.get(
            f'https://elite.finviz.com/export.ashx?v=152&t={",".join(tickers)}&o=-change&c=1,2,3,5,6,7,75,16,82,78,28,38,39,40,41,42,44,46,48,54,65,66&auth={settings.FINVIZ_API_KEY}'
        )
        df = pd.read_csv(
            StringIO(await data.text())
        ).set_index('Ticker')
        return df.to_dict('index')

def get_performance(dataset: pd.DataFrame, weights: dict):
    returns = dataset[weights.keys()].pct_change()
    portfolio_returns = returns.dot(pd.Series(weights))
    return ((1 + portfolio_returns).cumprod() * 100).dropna()

